# Measuring and Modeling Individual Preferences (Python)
## Traditional Conjoint Analysis (Python)

In [1]:
#prepare for Python version 3x features and functions
from __future__ import division,print_function

In [3]:
# import packages for analysis and modeling
import pandas as pd 

# data frame operations
import numpy as np 

# arrays and math functions

import statsmodels.api as sm 

# statistical models(including regression)
import statsmodels.formula.api as smf 

# Rlike model specification
from patsy.contrasts import Sum

import warnings
warnings.filterwarnings("ignore")

In [3]:
# read in conjoint survey profiles with respondent ranks
conjoint_data_frame = pd.read_csv(r"C:\Users\Zachary_B\Desktop\dataset\mobile_services_ranking.csv")

In [4]:
# set up sum contrasts for effects coding as needed for conjoint analysis
# using C(effect, Sum) notation within maineffects model specification
main_effects_model = r"ranking ~ C(brand, Sum)+ C(startup, Sum) + C(monthly, Sum) + C(service, Sum) + C(retail, Sum) + C(apple, Sum) + C(samsung, Sum) + C(google, Sum)"

In [5]:
# fit linear regression model using main effects only (no interaction terms)
main_effects_model_fit =     smf.ols(main_effects_model, data = conjoint_data_frame).fit()
print(main_effects_model_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                ranking   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     97.07
Date:                Mon, 22 Aug 2022   Prob (F-statistic):             0.0794
Time:                        12:03:09   Log-Likelihood:                 10.568
No. Observations:                  16   AIC:                             8.864
Df Residuals:                       1   BIC:                             20.45
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

C:\Users\Zachary_B\anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [6]:
conjoint_attributes = ['brand', 'startup','monthly', 'service', 'retail', 'apple', 'samsung', 'google']

In [9]:
# build part-worth information one attribute at a time
level_name = []
part_worth = []
part_worth_range = []
end = 1 # initialize index for coefficient in params

for item in conjoint_attributes:
    nlevels = len(list(set(conjoint_data_frame[item])))
    level_name.append(list(set(conjoint_data_frame[item])))
    begin = end
    end = begin + nlevels - 1
    new_part_worth = list(main_effects_model_fit.params[begin:end])
    new_part_worth.append((-1) * sum(new_part_worth))
    part_worth_range.append(max(new_part_worth) - min(new_part_worth))
    part_worth.append(new_part_worth)
    # end set to begin next iteration

In [11]:
# compute attribute relative importance values from ranges
attribute_importance = []
for item in part_worth_range: 
    attribute_importance.append(round(100 *(item / sum(part_worth_range)),2))

In [12]:
# user-defined dictionary for printing descriptive attribute names
effect_name_dict = {'brand' : 'Mobile Service Provider', 'startup' : 'Start-up Cost', 'monthly' : 'Monthly Cost', 'service' : 'Offers 4G Service',
'retail': 'Has Nearby Retail Store', 'apple' : 'Sells Apple Products', 'samsung' : 'Sells Samsung Products', 'google' : 'Sells Google/Nexus Products'}

In [15]:
# report conjoint measures to console
index = 0 # initialize for use in for-loop
for item in conjoint_attributes:
    print('\nAttribute:',
effect_name_dict[item])
    print(' Importance:',
attribute_importance[index])
    print(' Level Part-Worths')
for level in range(len(level_name[index])):
    print(' ',level_name[index]
[level], part_worth[index][level])
index = index + 1


Attribute: Mobile Service Provider
 Importance: 2.38
 Level Part-Worths

Attribute: Start-up Cost
 Importance: 2.38
 Level Part-Worths

Attribute: Monthly Cost
 Importance: 2.38
 Level Part-Worths

Attribute: Offers 4G Service
 Importance: 2.38
 Level Part-Worths

Attribute: Has Nearby Retail Store
 Importance: 2.38
 Level Part-Worths

Attribute: Sells Apple Products
 Importance: 2.38
 Level Part-Worths

Attribute: Sells Samsung Products
 Importance: 2.38
 Level Part-Worths

Attribute: Sells Google/Nexus Products
 Importance: 2.38
 Level Part-Worths
  "Verizon" 8.881784197001252e-15
  "AT&T" -0.25000000000001266
  "T-Mobile" 5.329070518200751e-15
  "US Cellular" 0.24999999999999845
